In [30]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
from ast import literal_eval
from time import gmtime, strftime
import re
import time
from bs4 import BeautifulSoup
#$from tqdm import tnrange, tqdm_notebook
from tqdm.auto import tqdm

# Scrapping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

# Error Handling
import socket
import urllib3
import urllib.request
from urllib.request import urlopen
from urllib.parse import quote_plus
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict

In [5]:
import os

print("Current working directory : ", os.getcwd())

Current working directory :  G:\내 드라이브\Mycelebs\glowpick_scraping


In [103]:
df_brd_path = os.getcwd() + '/' + 'brd_info.csv'
df_brd = pd.read_csv(df_brd_path)
df_brd = df_brd.iloc[:, 1:]
brd_names = df_brd.loc[:, 'brand_name']

print("length of list of unique brand name : ", len(df_brd))
print("df_brd ⬇" )
df_brd[0:3]

length of list of unique brand name :  8052
df_brd ⬇


,brand_code,brand_name,category
0,https://www.glowpick.com/brands/8016,가가알로에,['스킨케어']
1,https://www.glowpick.com/brands/1265,가그린,['생활용품']
2,https://www.glowpick.com/brands/2077,가네보,"['페이스메이크업', '립메이크업', '컨투어링', '클렌징', '배쓰&바디', '..."


In [20]:
v4_path = os.getcwd() + '/' + 'V4_DF_ind_set.csv'
df_v4 = pd.read_csv(v4_path)

print('length of item list of v4: ', len(df_v4))
print("df_v4 ⬇" )
df_v4[0:3]

length of item list of v4:  75121
df_v4 ⬇


,브랜드_link,브랜드명,브랜드_id,제품_link,제품_id,제품명,제품 용량,제품 가격,유사 제품군 랭킹(int),유사 제품군 랭킹(text),...,제품 성분 구성 : 중간 위험도(갯수),제품 성분 구성 : 중간 위험도 (성분(한글)),제품 성분 구성 : 중간 위험도 (성분(영어)),제품 성분 구성 : 중간 위험도 (성분(설명)),제품 성분 구성 : 높은 위험도,제품 성분 구성 : 높은 위험도(갯수),제품 성분 구성 : 높은 위험도 (성분(한글)),제품 성분 구성 : 높은 위험도 (성분(영어)),제품 성분 구성 : 높은 위험도 (성분(설명)),제품 카테고리
0,,0720,NaN,https://www.glowpick.com/products/89920,89920,오일크러쉬 팩트,8g,"7,500원",115위,프레스드파우더,...,3개,"['미네랄오일', '디메치콘;다이메티콘', '피티에프이']","['MineralOil;ParaffinumLiquidum', 'Dimethicone...","['착향제,헤어컨디셔닝제,피부보호제,피부컨디셔닝제(유연제),피부컨디셔닝제(수분차단제...",높은위험도,0개,[],[],[],"['페이스메이크업', '피니시파우더', '프레스드파우더']"
1,,0720,NaN,https://www.glowpick.com/products/104423,104423,봄봄 반달형 휴대용 브러시,1p,"2,000원",151위,파우더브러시,...,0개,[],[],[],높은위험도,0개,[],[],[],"['뷰티툴', '브러시', '파우더브러시']"
2,,0720,NaN,https://www.glowpick.com/products/97152,97152,[단종] 걸크러쉬 마스카라,1p,"2,000원",[],[],...,0개,[],[],[],높은위험도,0개,[],[],[],"['아이메이크업', '마스카라', '볼륨마스카라']"


In [10]:
print('lenght of unique brand name in v4: ', len(list(set(df_v4.브랜드명))))

lenght of unique brand name in v4:  4134


# 브랜드별 카테고리 수집

In [20]:
#브랜드별 카테고리 수집

wd = webdriver.Chrome(ChromeDriverManager().install())
categories = []
idx = 0
total = len(df_brd)

for i in df_brd.loc[:, 'brand_code']:
    try:
        category = []
        wd.get(i)
        time.sleep(2)
        wd.find_element_by_xpath('//*[@id="default-layout"]/div/div[1]/span/div/div[2]/div[2]/button[2]').click()
        time.sleep(2)

        html = wd.page_source
        soup = soup = BeautifulSoup(html,"html.parser")

        for i in range(len(soup.findAll('div', {'class':'selector__item__div'}))):
            category_name = soup.select('.selector__item__div')[i].get_text().strip()
            category.append(category_name)
        category.pop(0) #'전체' 카테고리 제외
        categories.append(category)
        idx += 1
        print(idx, '/ ',total)
        

    except Exception as e:
        print(idx, '번 오류 : ', e)
        categories.append([])
        error.append(idx)
        idx += 1
        pass



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/kimjisoo/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


1 /  8052
2 /  8052
3 /  8052
4 /  8052
5 /  8052
6 /  8052
7 /  8052
8 /  8052
9 /  8052
10 /  8052
11 /  8052


KeyboardInterrupt: 

# 브랜드별 제품 link, name 수집

In [89]:
dict_prd_link = {}
dict_prd_name = defaultdict(list)
no_such_element_exception = []
unknown_exception = []

In [123]:
def scrap_product_link(brd_name):
    
    try: 
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        print(userAgent)

        options.add_argument('headless')
        options.add_argument('window-size=1920x1080')
        options.add_argument("disable-gpu")
        options.add_argument(f'user-agent={userAgent}')
        wd = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        socket.setdefaulttimeout(30)

        url = 'https:/www.glowpick.com/searches/keywords'
        wd.get(url)
        time.sleep(5)

        wd.find_element_by_xpath("//*[@id='default-layout']/div/div[1]/span/div/div[2]/div[2]/button[2]").click()
        wd.maximize_window()
        time.sleep(3)

        brd_input = wd.find_element_by_xpath("//*[@id='default-layout']/div/div[2]/div/div/div/input")
        brd_input.send_keys(brd_name)
        brd_input.send_keys(Keys.ENTER)
        time.sleep(3)

        def parsing_html_with_class(parsing_info = '.products'):
            html = wd.page_source
            soup = soup = BeautifulSoup(html, "html.parser")
            interm_r = soup.select(parsing_info)
            return interm_r

        try:
            while True:
                prd_link = []
                prd_name = []

                product_lst = parsing_html_with_class()
                product_len = len(re.findall('<div class="products__product product" (.*?)', str(product_lst[0])))
                
                #while wd.find_elements_by_class_name('details__brand')[product_len].text[:len(brd_name)] == brd_name:

                element = wd.find_element_by_xpath('/html/body/div/div/div/div/main/div/section/div[2]/div/div[1]/span/div[{}]'.format(product_len - 1))
                wd.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(2)

                product_lst_2 = parsing_html_with_class()

                if product_lst == product_lst_2:
                    for i in range(product_len):
                        if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                            code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                            link = 'https://www.glowpick.com/products/' + code
                            prd_link.append(link)
                            dict_prd_name[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
                    dict_prd_link[brd_name] = prd_link
                    break

        except NoSuchElementException:
            for i in range(product_len):
                if (re.sub(r'\([^)]*\)', '', wd.find_elements_by_class_name('details__brand')[i].text).strip()) == brd_name:
                    code = re.findall('<div class="products__product product" (.*?)id="(.*?)">', str(product_lst[0]))[i][1].split('-')[-1]
                    link = 'https://www.glowpick.com/products/' + code
                    prd_link.append(link)
                    dict_prd_name[brd_name].append(wd.find_elements_by_class_name('details__product')[i].text)
            dict_prd_link[brd_name] = prd_link
        
    except NoSuchElementException:
        no_such_element_exception.append(brd_name)
        print("brand name: {}".format(brd_name), "NoSuchElementException")
        pass

    except Exception as e:
        unknown_exception.append(brd_name)
        print("brand name: {}".format(brd_name), e)
        pass
    

In [125]:
for i in trange(len(brd_names[100:300])):
    brd_name = df_brd.loc[i+100, 'brand_name']
    scrap_product_link(brd_name)

  0%|          | 0/200 [00:00<?, ?it/s]



====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36


...

Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


====== WebDriver manager ======


Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1468.0 Safari/537.36


Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\keunb\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [147]:
df_brd['product_link'] = df_brd['brand_name'].map(dict_prd_link)
df_brd['product_name'] = df_brd['brand_name'].map(dict_prd_name)

df_brd.to_csv("df_brd_prd_0_300.csv")

In [121]:
brd_names[50:100]

50         갯츠맨
51        갸마르드
52         갸스비
53       거다슈필만
54      건강한형제들
55       건영크린텍
56    걸 드 프로방스
57         걸리걸
58          검염
59         게리쏭
60          고원
61         고위드
62        고위드미
63         고유에
64         고은재
65         고혼진
66       골드쉐이프
67       골든 허브
68        골든트리
69          공미
70         공스킨
71         공일오
72        과일나라
73        광동제약
74        교보문고
75         구구샵
76       구닥 컬러
77          구달
78          구디
79       구딸 파리
80          구떼
81        구뜨앙쥬
82          구멍
83       구미올로지
84         구미킹
85         구본초
86        구스크릭
87          구찌
88       국대마스크
89       국보마스크
90        국제약품
91        굿락투유
92         굿매너
93       굿바이아톱
94       굿스킨랩스
95         굿앤닥
96         굿햄프
97          궁가
98         궁백화
99        궁중비책
Name: brand_name, dtype: object

In [122]:
df_brd.loc[60, 'product_name']

['발효 다슬 에센스', '호랑이 수딩젤', '호랑이 스킨토너']

In [126]:
unknown_exception

['겔랑', '글로우']

In [127]:
no_such_element_exception

['가란시아', '게리쏭']

In [60]:
df_prd_link = pd.DataFrame.from_dict(dict_prd_link, orient='index')

In [143]:
list(dict_prd_link.items())[292]

('나트라케어',
 ['https://www.glowpick.com/products/73824',
  'https://www.glowpick.com/products/73826',
  'https://www.glowpick.com/products/73823',
  'https://www.glowpick.com/products/73833',
  'https://www.glowpick.com/products/102827',
  'https://www.glowpick.com/products/73832',
  'https://www.glowpick.com/products/73835',
  'https://www.glowpick.com/products/74226',
  'https://www.glowpick.com/products/96014',
  'https://www.glowpick.com/products/73840',
  'https://www.glowpick.com/products/73842',
  'https://www.glowpick.com/products/96019',
  'https://www.glowpick.com/products/96017',
  'https://www.glowpick.com/products/74228'])

In [146]:
list(dict_prd_name.items())[293]

('나투라비타',
 ['마더스 스킨케어 오일',
  '베이비 굿데이 마일드 선크림 [SPF40/PA++]',
  '마더스 스킨케어 크림',
  '베이비 러브 바디 로션',
  '베이비 러브 헤어 앤 바디 솝'])

In [134]:
len(dict_prd_name.keys())

8033

In [136]:
dict_prd_name.keys()

dict_keys(['가가알로에', '가그린', '가네보', '가드맨', '가드미', '가든 오브 라이프', '가디스가든', '가르니르', '가르니에', '가오쿠', '가온', '가온도담', '가이아', '가이아 허브', '가인비책', '가쯔', '가히', '간나나기', '간만세', '게볼', '게스', '게스통', '겐조', '겐조키', '겐지코', '겔라시앙', '겟뉴스킨', '겟츠', '결고은사람들', '경남제약', '경동제약', '경인 플랙스', '고귀한아름다움채', '고려생활건강', '고려은단', '고사리숲', '고세', '고스트', '갈랑트리', '갈리니', '갈리마드', '감람나무', '갓스킨', '갓트', '강남이스트', '강블리', '강서화학', '갤럭시본', '갯츠맨', '갸마르드', '갸스비', '거다슈필만', '건강한형제들', '건영크린텍', '걸 드 프로방스', '걸리걸', '검염', '고원', '고위드', '고위드미', '고유에', '고은재', '고혼진', '골드쉐이프', '골든 허브', '골든트리', '공미', '공스킨', '공일오', '과일나라', '광동제약', '교보문고', '구구샵', '구닥 컬러', '구달', '구디', '구딸 파리', '구떼', '구뜨앙쥬', '구멍', '구미올로지', '구미킹', '구본초', '구스크릭', '구찌', '국대마스크', '국보마스크', '국제약품', '굿락투유', '굿매너', '굿바이아톱', '굿스킨랩스', '굿앤닥', '굿햄프', '궁가', '궁백화', '궁중비책', '가란시아', '겔랑', '게리쏭', '권도영알로에', '그나랜', '그날레시피', '그라니트', '그라시아', '그라운드랩', '그라운드플랜', '그라티아수', '그라펜', '그라함스', '그란디디에', '그란츠', '그랑핸드', '그래잇포유', '그랜드파', '그램', '그레노스', '그레이그라운드', '그레이드', '그레이멜린', '그레이스데이', '그레이스유', '그로가닉스', '그로우어스', '그로운 알케미스트', '그루

In [1]:
import ast

In [33]:
import pickle
os.getcwd()

'/Users/kimjisoo/Desktop/preprocessing'

In [35]:
with open('dict_prd_link.txt', 'wb') as fp:
    pickle.dump(dict_prd_link, fp)

In [37]:
dict_prd_name

{'결고은사람들': ['어성초 크림', '[단종]  클렌징 오일'],
 '경남제약': ['결콜라겐',
  '생유산균9C',
  '생 유산균 알파',
  '레모나 마스크 KF-AD',
  '결콜라겐 플러스',
  '결콜라겐&뮤신',
  '스트레스 케어',
  '생유산균7C',
  '칼로 스탑',
  '핸드 클린 손 소독제',
  '면역 부스터',
  '루테인 톡',
  '칼로-뷰티 콜라겐 1000'],
 '경동제약': ['애니밴드(원형)'],
 '경인 플랙스': ['황사 방역용 마스크 KF94', '황사 방역용 마스크 KF80'],
 '고귀한아름다움채': ['어성초 황금비율 두피앰플', '지청 자양 샴푸액'],
 '고려생활건강': ['티엠씨 에스 바'],
 '고려은단': ['비타민C 1000',
  '비타민C 골드 플러스',
  '비타민C 1000 이지',
  '메가도스B',
  '메가도스 C3000',
  '퓨어 알티지 오메가3',
  '비타1000 마스크팩',
  '신바이오틱스',
  '다이어트 쉐이크',
  '루테인 지아잔틴 아스타잔틴',
  '비타미녀씨',
  '퓨어 밀크씨슬',
  '퓨어 루테인',
  '프리바이오틱스',
  '듀얼 플로라 프로바이오틱스',
  '알티지 오메가3',
  '퓨어 루테인 지아잔틴'],
 '고사리숲': ['[단종]  꼬미 주니어 클렌징 폼'],
 '고세': ['클리어턴 에센스 마스크팩 - 비타민 C',
  '쓰리버터스 립 밤',
  '에스프리끄 포밍 뷰티 리퀴드 UV 파운데이션 [SPF23/PA++]',
  '프레디아 스파에메르 헤드 스파클링 에센스',
  '모이스처 스킨 리페어 에센스',
  '클리어턴 에센스 마스크팩 - 히알루론산',
  '모이스처 스킨 리페어 아이 크림',
  '프레디아 스파에메르 G 세럼',
  '클리어턴 에센스 마스크팩 - 콜라겐',
  '클리어턴 에센스 마스크팩 - 프라센타',
  '클리어턴 에센스 마스크팩 - 코엔자임 Q10',
  '클리어턴 에센스 마스크팩 - 트라네키 삼산',
  '스펀지 크리너